In [25]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table

engine=create_engine('postgresql://postgres:<password>@solarmsuproject.cfrh4os4lzss.us-east-1.rds.amazonaws.com:5432/msuproject')

In [26]:
metadata=MetaData()
print(metadata.tables)

FacadeDict({})


In [27]:
metadata.reflect(bind=engine)
print(metadata.tables)

FacadeDict({'alpena_county': Table('alpena_county', MetaData(), Column('year', INTEGER(), table=<alpena_county>, nullable=False), Column('month', INTEGER(), table=<alpena_county>, nullable=False), Column('day', INTEGER(), table=<alpena_county>, nullable=False), Column('hour', INTEGER(), table=<alpena_county>, nullable=False), Column('minute', INTEGER(), table=<alpena_county>, nullable=False), Column('dhi', DOUBLE_PRECISION(precision=53), table=<alpena_county>, nullable=False), Column('dni', DOUBLE_PRECISION(precision=53), table=<alpena_county>, nullable=False), Column('ghi', DOUBLE_PRECISION(precision=53), table=<alpena_county>, nullable=False), Column('clearsky_dhi', DOUBLE_PRECISION(precision=53), table=<alpena_county>, nullable=False), Column('clearsky_dni', DOUBLE_PRECISION(precision=53), table=<alpena_county>, nullable=False), Column('clearsky_ghi', DOUBLE_PRECISION(precision=53), table=<alpena_county>, nullable=False), Column('cloud_type', DOUBLE_PRECISION(precision=53), table=<a

In [28]:
solar_data=metadata.tables['complete_deep_solar_and_demographics_mich']
print(solar_data)

complete_deep_solar_and_demographics_mich
